# Airbnb NYC
- https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data

- http://insideairbnb.com/get-the-data.html

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nyc = pd.read_csv('AB_NYC_2019.csv')

In [3]:
# 20 April, 2020 Data from airbnb.com
#aus = pd.read_csv('austin_listings.csv')

In [4]:
# 14 April, 2020 Data from airbnb.com
#bos = pd.read_csv('boston_listings.csv')

In [5]:
# 08 April, 2020 Data from airbnb.com
#nyc = pd.read_csv('nyc_listings.csv')

In [6]:
nyc.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [7]:
#aus.head(5)

In [8]:
#aus.info()

In [9]:
#bos.info()

In [10]:
nyc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
id                                48895 non-null int64
name                              48879 non-null object
host_id                           48895 non-null int64
host_name                         48874 non-null object
neighbourhood_group               48895 non-null object
neighbourhood                     48895 non-null object
latitude                          48895 non-null float64
longitude                         48895 non-null float64
room_type                         48895 non-null object
price                             48895 non-null int64
minimum_nights                    48895 non-null int64
number_of_reviews                 48895 non-null int64
last_review                       38843 non-null object
reviews_per_month                 38843 non-null float64
calculated_host_listings_count    48895 non-null int64
availability_365                  48895 non-null int64

In [11]:
# Checking for null values 
nyc.isnull().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [14]:
# Dropping identifier columns - host_name, name of listing
nyc.drop(['name', 'host_name'], axis = 1,inplace = True)

In [23]:
nyc.isnull().sum()

id                                0
host_id                           0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [19]:
nyc[['reviews_per_month','last_review']]

,reviews_per_month,last_review
0,0.21,2018-10-19
1,0.38,2019-05-21
2,NaN,NaN
3,4.64,2019-07-05
4,0.10,2018-11-19
...,...,...
48890,NaN,NaN
48891,NaN,NaN
48892,NaN,NaN
48893,NaN,NaN


In [20]:
# dropping last review column as it is the date of last review.
nyc.drop(['last_review'], axis = 1,inplace = True)

In [22]:
# replacing null values in reviews per month to 0. This is a probably the rating
nyc.fillna({'reviews_per_month':0}, inplace=True)

In [25]:
nyc.columns

Index(['id', 'host_id', 'neighbourhood_group', 'neighbourhood', 'latitude',
       'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

In [33]:
nyc[['price', 'minimum_nights',
       'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365']].describe()

,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,48884.000000,48884.000000,48884.000000,48884.000000,48884.000000,48884.000000
mean,152.755053,7.029887,23.271991,1.090800,7.144628,112.779498
std,240.170260,20.512224,44.551331,1.597213,32.956185,131.627271
min,10.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,69.000000,1.000000,1.000000,0.040000,1.000000,0.000000
50%,106.000000,3.000000,5.000000,0.370000,1.000000,45.000000
75%,175.000000,5.000000,24.000000,1.580000,2.000000,227.000000
max,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [31]:
(nyc['price'] ==0).value_counts()

False    48884
True        11
Name: price, dtype: int64

In [32]:
#deleting listings with price = 0. Price should be greater than 0
nyc.drop(nyc.loc[nyc['price']==0].index, inplace=True)

In [36]:
nyc[['neighbourhood_group','neighbourhood','room_type']].astype('object').describe()

,neighbourhood_group,neighbourhood,room_type
count,48884,48884,48884
unique,5,221,3
top,Manhattan,Williamsburg,Entire home/apt
freq,21660,3919,25407


In [37]:
nyc.host_id.nunique()

37455

In [43]:
nyc.host_id.value_counts()

219517861    327
107434423    232
30283594     121
137358866    103
16098958      96
            ... 
1641589        1
4070519        1
208106618      1
235939247      1
1288080        1
Name: host_id, Length: 37455, dtype: int64